# What's in this notebook? 📓
Here, I scrape the video ids from a playlist of 786 commencement speeches, then use an API to get the transcripts. Then, I scrape NPR's list of top commencement speeches (on which they have trancriptions), and another webite with 45 more commencement speeches (also with transcripts). 

## Scraping for video ids 🎥
I used selenium to scrape the page with all of my playlist's videos. I could've dealt with figuring out how to get selenium to scroll for me, but out of exhaustion (I had just watched over 900 snippets of commencements, after all 😴), I just scrolled to get all 786 videos shown at once, which took all of 5 seconds. 

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [7]:
browser = webdriver.Chrome(executable_path="/Users/hannah/Downloads/chromedriver")
browser.get('https://www.youtube.com/playlist?list=PL0LlgJiV1zg27lYnrC8aE69o4ITvp-gAM')

In [10]:
html = browser.page_source

In [12]:
from bs4 import BeautifulSoup

In [13]:
soup = BeautifulSoup(html, 'html.parser')

In [34]:
to_loop_thru = soup.find_all('a', {'class':'yt-simple-endpoint style-scope ytd-playlist-video-renderer'})
ids = [to_loop_thru[i]['href'][9:20] for i in range(len(to_loop_thru))]

In [35]:
ids

['bPv21OyQLkM',
 'ngzIkKtjT6o',
 'y5YvCbOmNxQ',
 'Q34H3As2QJA',
 'abo-YcLrnao',
 'jsHBI4w4FgQ',
 '9ZDuNzhelhQ',
 '9LheUWrXUHU',
 'TvI69F8uf1A',
 '-vsLRM4Tjd0',
 'wQ4A8NLMAbY',
 'ArmeLya_pY8',
 'RIS2dlnlsRo',
 '907yEkALaAY',
 'QOopQWWfRXA',
 '0TgLYLuDjlY',
 'erLem1Zijmo',
 'MYOEAVlRy6k',
 'kHU8aG1xi_M',
 'SflExy5Lxzo',
 'JRUOZgc3WL0',
 'fqRgtaRMGtw',
 'XMbt090GMRs',
 'SAJH03U7aHM',
 'GUCy75CA3Aw',
 'oo18EZsVgUY',
 'LN2ZlDdaOA8',
 'IeKxySlQqcg',
 'WiWLAEY4XK0',
 'e0SsSr9Trro',
 'e4xSoj369pk',
 'UF8uR6Z6KLc',
 '8CrOL-ydFMI',
 'JTtE6j4TALM',
 'kvsJSEfhI6g',
 'HNyNlXZKEu8',
 '28-jRr9MMcY',
 'IulvPqb1Eus',
 'CS05u4FVeww',
 '69fQ8NMiwjA',
 '4E6ui2HsjjU',
 '0oBTWA4KU20',
 'jkollMFXnyQ',
 'S7IvGwEGH-4',
 '1BVmRt3ap0o',
 'ZzBCv8VoN3I',
 '_Rthrf43rIo',
 'U5wQdJEGZB4',
 'i5ZeIVPe5dg',
 'J7iYuWXFQz8',
 'ev3v0bAd1_w',
 'mJXqQHZj9YI',
 'YOBK-xBOFcc',
 'KFcBDoXKlD4',
 'qWQG8aE8o7s',
 'LOwm4zBg0sE',
 '-iTizg8Q0s4',
 'weQVu8rjrmA',
 'jsDVzNOeu48',
 'hrGZl5-3_zM',
 'TI7WIXW0no0',
 '7AchBTlFt_4',
 'tjb_iK

In [36]:
browser.close()

Great! We have all the ids. Now, we can use the pypi library youtube-transcript-api to get the transcripts! 👏

In [41]:
from youtube_transcript_api import YouTubeTranscriptApi
import time
from tqdm import tqdm_notebook

raw_transcripts = []
for video_id in tqdm_notebook(ids):
    raw_transcripts.append(YouTubeTranscriptApi.get_transcript(video_id))
    time.sleep(1)

In [43]:
len(raw_transcripts)

786

Hooray! We have all the transcripts! However, they're in a bit of an awkward format, as lists of dictionaries that break down the whole transcript into segments. So, we'll join those all together into one long string, and also link the transcripts back up with their video ids (just in case we ever want to go back and look at the commencement address being delivered).